<a href="https://colab.research.google.com/github/botatooo/pp-detection-fracture-recherche/blob/dev/src/fracatlas_efficientdet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'


In [2]:
import torch

!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]

print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.1 ; cuda:  cu121


In [3]:
import os
import json
from torchvision.datasets.utils import download_and_extract_archive

root = "dataset/"
url = "https://figshare.com/ndownloader/files/41725659"
filename = "fracatlas.zip"

# if download:
if not os.path.isdir(os.path.join(root, "FracAtlas")):
    os.makedirs(root, exist_ok=True)
    download_and_extract_archive(
        url,
        os.path.dirname(root),
        filename=filename,
        remove_finished=True,
    )
if not os.path.isdir(root):
    raise RuntimeError(
        "Dataset not found or corrupted. You can use download=True to download it"
    )

with open("dataset/FracAtlas/Annotations/COCO JSON/COCO_fracture_masks.json") as f:
  fracture_masks_data = json.load(f)

fractured_images = [i["file_name"] for i in fracture_masks_data["images"]]
fractured_image_count = len(fractured_images)

training_images = fracture_masks_data["images"][: int(0.9 * fractured_image_count)]
testing_images = fracture_masks_data["images"][int(0.9 * fractured_image_count) :]

os.mkdir("data")


os.mkdir("data/images")
os.mkdir("data/images/train2017")
for i in training_images:
  full_path = os.path.abspath(os.path.join("dataset/FracAtlas/images/Fractured", i["file_name"]))
  new_path = os.path.abspath(os.path.join("data/images/train2017", i["file_name"]))
  os.rename(full_path, new_path)

os.mkdir("data/images/val2017")
for i in testing_images:
  full_path = os.path.abspath(os.path.join("dataset/FracAtlas/images/Fractured", i["file_name"]))
  new_path = os.path.abspath(os.path.join("data/images/val2017", i["file_name"]))
  os.rename(full_path, new_path)

os.mkdir("data/annotations")
with open("data/annotations/instances_train2017.json", "w") as f:
  json.dump({
      'info': {'description': 'my-project-name'},
      'images': training_images,
      'categories': [{'id': 0, 'name': 'fractured'}]
  }, f)
with open("data/annotations/instances_val2017.json", "w") as f:
  json.dump({
      'info': {'description': 'my-project-name'},
      'images': testing_images,
      'categories': [{'id': 0, 'name': 'fractured'}]
  }, f)


100%|██████████| 338412751/338412751 [00:03<00:00, 99813029.28it/s] 


Extracting dataset/fracatlas.zip to dataset


In [4]:
!git clone https://github.com/toandaominh1997/EfficientDet.Pytorch efficientdet
%cd efficientdet

Cloning into 'efficientdet'...
remote: Enumerating objects: 791, done.
remote: Total 791 (delta 0), reused 0 (delta 0), pack-reused 791
Receiving objects: 100% (791/791), 11.21 MiB | 18.91 MiB/s, done.
Resolving deltas: 100% (460/460), done.
/content/efficientdet


In [5]:
!python train.py --help

usage: train.py [-h] [--dataset {VOC,COCO}] [--dataset_root DATASET_ROOT] [--network NETWORK]
                [--resume RESUME] [--num_epoch NUM_EPOCH] [--batch_size BATCH_SIZE]
                [--num_class NUM_CLASS] [--device DEVICE]
                [--grad_accumulation_steps GRAD_ACCUMULATION_STEPS] [--lr LR]
                [--momentum MOMENTUM] [--weight_decay WEIGHT_DECAY] [--gamma GAMMA]
                [--save_folder SAVE_FOLDER] [-j N] [--start_epoch N] [--world-size WORLD_SIZE]
                [--rank RANK] [--dist-url DIST_URL] [--dist-backend DIST_BACKEND] [--seed SEED]
                [--gpu GPU] [--multiprocessing-distributed]

PyTorch ImageNet Training

options:
  -h, --help            show this help message and exit
  --dataset {VOC,COCO}  VOC or COCO
  --dataset_root DATASET_ROOT
                        Dataset root directory path [/root/data/VOCdevkit/, /root/data/coco/]
  --network NETWORK     efficientdet-[d0, d1, ..]
  --resume RESUME       Checkpoint state_dict fi

In [6]:
!sed -i 's|http://storage.googleapis.com/public-models/efficientnet/efficientnet-b0-355c32eb.pth|https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth|g' models/utils.py
!python train.py --dataset COCO --dataset_root /content/data --network efficientdet-d0 --batch_size 32

/content/efficientdet/train.py:302: UserWarning: You have chosen to seed training. This will turn on the CUDNN deterministic setting, which can slow down your training considerably! You may see unexpected behavior when restarting from checkpoints.
  warnings.warn('You have chosen to seed training. '
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Downloading: "https://github.com/lukemelas/EfficientNet-Py